# Gerando conjunto de treinamentos

In [ ]:
import rasterio
from rasterio import features
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import box
import glob
import re

In [ ]:
scm_licoes = pd.read_csv('downloads/imagens/SCMs-licoes.csv')
licoes_utilizadas = scm_licoes.loc[scm_licoes["presente"] == 1, 'scm']
licoes = gpd.read_file('licoes/licoes-por-scm.gpkg')
licoes = licoes[licoes.scm.isin(licoes_utilizadas)]
gdf_quadras = gpd.read_file('zip://downloads/SIRGAS_SHP_quadraMDSF.zip!/SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp')

In [ ]:
tamanho_janela = 24 # tamanho em pixels
sobreposicao_de_janelas = 16 # sobreposição das janelas em pixels

In [ ]:
 

for f in glob.glob("pre-process/grayscales/*.jp2"):
    
    data, target = [], []

    dataset = rasterio.open(f)
    scm = re.search('gray-scale-(.*).jp2', f).group(1)
    
    grayscale = dataset.read(1).astype('float32')

    labels = features.rasterize(
        ((g, 255) for g in licoes[licoes.scm == scm].geometry.explode()),
        out_shape=dataset.shape,
        transform=dataset.transform)

    quadras_scm = gpd.clip(gdf_quadras, gpd.GeoSeries(box(*dataset.bounds))).unary_union

    quadras = features.rasterize(
        ((g, 255) for g in quadras_scm),
        out_shape=dataset.shape,
        transform=dataset.transform)

    hog_image = np.load(f'pre-process/hogs/hog-{scm}.npy').astype('float32')
    lbp = np.load(f'pre-process/hogs/hog-{scm}.npy').astype('float32')

    rua = 0
    faixa = 0

    for i in np.arange(0, grayscale.shape[0] - tamanho_janela, tamanho_janela - sobreposicao_de_janelas):
        for j in np.arange(0, grayscale.shape[1] - tamanho_janela, tamanho_janela - sobreposicao_de_janelas):
            # Se mais de 80% dos pixels estiverem fora da quadra
            if np.count_nonzero(quadras[i:i+tamanho_janela, j:j+tamanho_janela] == 255) < 0.05 * (tamanho_janela ** 2):
                # print(i, j)
                # Se mais de 80% dos pixels estiverem dentro de um label de faixa de pedestre
                img_gray = grayscale[i:i+tamanho_janela, j:j+tamanho_janela]
                img_hog = hog_image[i:i+tamanho_janela, j:j+tamanho_janela]
                img_lbp = lbp[i:i+tamanho_janela, j:j+tamanho_janela]

                # Data Augmentation
                
                img = np.concatenate([
                    img_gray,
                    img_hog,
                    img_lbp
                ]).reshape(1, 3 * (tamanho_janela ** 2))
                
                if np.count_nonzero(labels[i:i+tamanho_janela, j:j+tamanho_janela] == 255) > 0.80 * (tamanho_janela ** 2):
                    faixa += 16
                    

                    for r in ['rotaciona 0', 'rotaciona 90', 'rotaciona 180', 'rotaciona 270']:
                        
                        # flip Up/Down

                        img_flipud = np.concatenate([
                            np.flipud(img_gray),
                            np.flipud(img_hog),
                            np.flipud(img_lbp)
                        ]).reshape(1, 3 * (tamanho_janela ** 2))

                        # flip Left/Right

                        img_fliplr = np.concatenate([
                            np.fliplr(img_gray),
                            np.fliplr(img_hog),
                            np.fliplr(img_lbp)
                        ]).reshape(1, 3 * (tamanho_janela ** 2))

                        # flip

                        img_flip = np.concatenate([
                            np.flip(img_gray),
                            np.flip(img_hog),
                            np.flip(img_lbp)
                        ]).reshape(1, 3 * (tamanho_janela ** 2))

                        for fl_img in (img, img_flipud, img_fliplr, img_flip):
                        # for fl_img in (img):
                            data.append(fl_img.reshape(1, 3 * (tamanho_janela ** 2)))
                            target.append(1)


                        img_gray, img_hog, img_lbp = np.rot90(img_gray), np.rot90(img_hog), np.rot90(img_lbp)

                else:
                    rua += 1
                    data.append(img.reshape(1, 3 * (tamanho_janela ** 2)))
                    target.append(0)

    np.save(f'pre-process/data/data-{scm}.npy', np.array(data).squeeze())
    np.save(f'pre-process/data/target-{scm}.npy', np.array(target))
    print(f"SCM: {scm}: {rua} ruas e {faixa} faixas")
    # break


In [ ]:
np.count_nonzero(np.array(target))